# lets create a class for the DUMMY LLM

In [2]:
import random

class MyFirstLLM:

  def __init__(self):
    print("LLM created")

  def predict(self, prompt):

    response_list = [
        "Kathmandu is the capital of NEPAL",
        "NPL is a cricket league",
        "AI stands for Artificial Intelligence"
    ]
    return { "response": random.choice(response_list)}

In [6]:
llm = MyFirstLLM()
llm.predict("what is the capital of nepal?")

LLM created


{'response': 'Kathmandu is the capital of NEPAL'}

# prompt template

In [13]:
class PromptTemplate:

  def __init__(self, template, input_variables):

    self.template = template
    self.input_variables = input_variables

  def format(self, input_dict):
    return self.template.format(**input_dict)

In [17]:
template = PromptTemplate(
    template="write a {length} poem about {topic}",
    input_variables=["topic", "length"]
)

In [20]:
prompt = template.format({"topic":"AI", "length":"short"})

In [21]:
llm = MyFirstLLM()

LLM created


In [24]:
llm.predict(prompt)

{'response': 'AI stands for Artificial Intelligence'}

# lets byuild a dupli llm chain

In [25]:
class DupliLLMChain:

  def __init__(self, llm , prompt):
    self.llm = llm
    self.prompt = prompt

  def run(self, input_dict):

    final_prompt = self.prompt.format(input_dict)
    result = self.llm.predict(final_prompt)


    return result['response']


In [26]:
template = PromptTemplate(
    template="write a {length} poem about {topic}",
    input_variables=["topic", "length"]
)

In [27]:
llm = MyFirstLLM()

LLM created


In [28]:
chain = DupliLLMChain(llm, template)

In [29]:
chain.run({"topic":"AI", "length":"short"})

'AI stands for Artificial Intelligence'

# lets solve this problem using abstractions ie making the standarization in chains

In [32]:
from abc import ABC , abstractmethod

In [33]:
class Runnable(ABC):

  @abstractmethod
  def invoke(self, input_data):
    pass

In [36]:
import random

class MyFirstLLM(Runnable):

  def __init__(self):
    print("LLM created")


  def invoke(self, prompt):
    response_list = [
        "Kathmandu is the capital of NEPAL",
        "NPL is a cricket league",
        "AI stands for Artificial Intelligence"
    ]
    return { "response": random.choice(response_list)}


  def predict(self, prompt):

    response_list = [
        "Kathmandu is the capital of NEPAL",
        "NPL is a cricket league",
        "AI stands for Artificial Intelligence"
    ]
    return { "response": random.choice(response_list)}

In [37]:
class PromptTemplate(Runnable):

  def __init__(self, template, input_variables):

    self.template = template
    self.input_variables = input_variables


  def invoke(self, input_dict):
    return self.template.format(**input_dict)

  def format(self, input_dict):
    return self.template.format(**input_dict)




In [48]:
class DupliOutputParser(Runnable):

  def __init__(self):
   pass

  def invoke(self, input_data):
    return input_data['response']

In [39]:
class RunnableConnector(Runnable):

  def __init__(self, runnable_list):
    self.runnable_list = runnable_list

  def invoke(self, input_data):
    for runnable in self.runnable_list:
      input_data = runnable.invoke(input_data)

    return input_data

In [40]:
template = PromptTemplate(
    template="write a {length} poem about {topic}",
    input_variables=["topic", "length"]
)

In [41]:
llm = MyFirstLLM()

LLM created


In [49]:
parser = DupliOutputParser()

In [51]:
chain = RunnableConnector([template, llm, parser])

In [52]:
chain.invoke({"length":"long", "topic":"Nepal"})

'Kathmandu is the capital of NEPAL'

In [53]:
#

In [54]:
template1 = PromptTemplate(
    template="write a Joke about {topic}",
    input_variables=["topic"]
)

In [55]:
template2 = PromptTemplate(
    template="Explain the following joke {response}",
    input_variables=["response"]
)

In [56]:
llm = MyFirstLLM()

LLM created


In [57]:
parser = DupliOutputParser()

In [58]:
chain1 = RunnableConnector([template1, llm,])

In [61]:
chain2 = RunnableConnector([template2, llm, parser])

In [63]:
chain2.invoke({"response":"this is a joke"})

'AI stands for Artificial Intelligence'

In [64]:
final_chain = RunnableConnector([chain1, chain2])

In [65]:
final_chain.invoke({"topic":"football"})

'AI stands for Artificial Intelligence'